## The CrossTree

Voting Schema with multiple binary classification trees. The network implements a voting scheme based on three different trees:

* Cultural **Agnostic-Rappresentative** tree
* Cultural **Agnostic-Exclusive** tree
* Cultural **Exclusive-Rappresentative** tree

the most voted class will be the predicted class.

### Training Phase

The training process is quite standard and straight-forward: given the n G_features we want to directly predict the associated class.

### Employment Phase

The training model will be inserted in a wider model called X and utilized as a function for the computation of the G_Factor

## Dataset

Load the dataset

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from CU_Dataset_Factory import Hf_Loader, CU_Dataset_Factory

def onehot_encode(
    df_train: pd.DataFrame,
    df_test: pd.DataFrame,
    cat_cols: list[str]|None = None,
    num_cols: list[str]|None = None,
    sparse: bool = False
) -> tuple[pd.DataFrame, pd.DataFrame, OneHotEncoder]:
    
    """
    Applies One-Hot Encoding to df_train and df_test guaranteeing the same
    set of columns, even if train is missing categories who are in the test set.

    Parameters
    ----------
    df_train : pd.DataFrame
        Training DataFrame.
    df_test : pd.DataFrame
        Testing DataFrame.
    cat_cols : list[str], optional
        List of categorical columns to encode.
        If None, all columns of type 'object' are taken.
    num_cols : list[str], optional
        List of numerical (or non-categorical) columns to preserve.
        If None, all columns not in cat_cols are taken. 
    handle_unknown : str, default="ignore"
        Beahavior on unknown values in test (typically "ignore").
    sparse : bool, default=False
        If True, returns sparse matrix, otherwise dense.

    Returns
    -------
    df_train_enc : pd.DataFrame
        Training DataFrame with One-Hot Encoding + original num_cols.
    df_test_enc : pd.DataFrame
        Testing DataFrame with One-Hot Encoding + original num_cols.
    encoder : OneHotEncoder
        The fitted OneHotEncoder object, useful for future transform.
    """
    
    # 1) Identify category and numerical columns (if not given)
    if cat_cols is None:
        cat_cols = df_train.select_dtypes(include="object").columns.tolist()
    if num_cols is None:
        num_cols = [c for c in df_train.columns if c not in cat_cols]

    # 2) Fit encoder on all category data (train + test)
    all_cats = pd.concat([df_train[cat_cols], df_test[cat_cols]], 
                         axis=0, ignore_index=True)
    encoder = OneHotEncoder(
        sparse_output=sparse
    ).fit(all_cats)

    # 3) Transform separatly train and test
    X_train_ohe = encoder.transform(df_train[cat_cols])
    X_test_ohe  = encoder.transform(df_test[cat_cols])

    # 4) Name the new columns
    ohe_cols = encoder.get_feature_names_out(cat_cols).tolist()

    # 5) Compose the final DataFrames
    df_train_enc = pd.DataFrame(
        np.hstack([X_train_ohe.toarray() if sparse else X_train_ohe,
                   df_train[num_cols].values]), # type: ignore
        columns=ohe_cols + num_cols,
        index=df_train.index
    )
    df_test_enc = pd.DataFrame(
        np.hstack([X_test_ohe.toarray() if sparse else X_test_ohe,
                   df_test[num_cols].values]),
        columns=ohe_cols + num_cols,
        index=df_test.index
    )

    return df_train_enc[ohe_cols], df_test_enc[ohe_cols], encoder


## Produce the Dataset

In [408]:
print('Cultural Dataset argumentation start')
factory = CU_Dataset_Factory('.')
train_l = Hf_Loader("sapienzanlp/nlp2025_hw1_cultural_dataset", 'train')
validation_l = Hf_Loader("sapienzanlp/nlp2025_hw1_cultural_dataset", 'validation')

factory.produce(train_l, 'train.tsv', ['category', 'subcategory','type','languages','reference','num_langs', 'ambiguos', 'G', 'n_mod', 'back_links'], 'label', 16, False)
factory.produce(validation_l, 'validation.tsv', ['category', 'subcategory','type','languages','reference','num_langs', 'ambiguos', 'G', 'n_mod', 'back_links'], 'label', 16, False)
print('End process')

Cultural Dataset argumentation start


 68%|██████▊   | 86/126 [00:02<00:01, 38.08it/s] 


KeyboardInterrupt: 

In [459]:
train = pd.read_csv('train.tsv', sep='\t')
validation = pd.read_csv('validation.tsv', sep='\t')

In [460]:
train.head(5)

,Unnamed: 0,back_links,n_mod,G_num_components,G_avg,G_nodes,G_largest_component_size,G_mean_pr,G_density,G_num_cliques,num_langs,reference,languages,type,subcategory,category,wiki_name,qid,label
0,0,36,79,1.0,1.000000,1.0,1.0,1.000000,0.000000,1.0,6,5,2,entity,film,films,916 (film),Q32786,1
1,1,222,614,1.0,1.417582,182.0,182.0,0.004405,0.013903,179.0,30,40,8,entity,musical group,music,!!!,Q371,2
2,2,24,16,1.0,3.200000,5.0,5.0,0.244156,0.900000,2.0,4,1,3,entity,comics,comics and anime,¡Soborno!,Q3729947,2
3,3,227,2375,1.0,1.455357,224.0,224.0,0.003656,0.011451,231.0,38,67,9,entity,musical group,music,+44 (band),Q158611,2
4,4,85,30,1.0,1.339623,159.0,159.0,0.004751,0.015206,153.0,2,7,1,entity,building,architecture,"1 Monk Street, Monmouth",Q280375,1


In [461]:
validation.head(5)

,Unnamed: 0,back_links,n_mod,G_num_components,G_avg,G_nodes,G_largest_component_size,G_mean_pr,G_density,G_num_cliques,num_langs,reference,languages,type,subcategory,category,wiki_name,qid,label
0,0,2582,784,1.0,1.000000,4.0,4.0,0.264605,0.500000,3.0,62,40,10,entity,sports club,sports,1. FC Nürnberg,Q15786,2
1,1,27,25,1.0,1.000000,1.0,1.0,1.000000,0.000000,1.0,7,4,4,entity,record label,music,77 Records,Q268530,1
2,2,841,2698,1.0,1.452888,329.0,329.0,0.002405,0.007525,343.0,68,166,10,entity,animated film,comics and anime,A Bug's Life,Q216153,2
3,3,21,36,1.0,1.000000,1.0,1.0,1.000000,0.000000,1.0,12,3,7,entity,film,films,A Gang Story,Q593,1
4,4,1940,1653,1.0,1.436170,188.0,188.0,0.004044,0.013369,177.0,60,91,10,entity,choreographer,performing arts,Aaron Copland,Q192185,2


In [462]:
y_train = train[['label']]
y_validation = validation[['label']]

id_train = train[['wiki_name']]
id_validation = validation[['wiki_name']]

fe_train = train[['languages', 'num_langs', 'reference','G_mean_pr']]
fe_validation = validation[['languages', 'num_langs', 'reference','G_mean_pr']]

fe_str_train = train[['category', 'subcategory', 'type']]
fe_str_validation = validation[['category', 'subcategory', 'type']]

In [463]:
train_cat, validation_cat, _ =  onehot_encode(fe_str_train, fe_str_validation, ['category'] )
train_scat, validation_scat, _ = onehot_encode(fe_str_train, fe_str_validation, ['subcategory'] )
train_t, validation_t, _ = onehot_encode(fe_str_train, fe_str_validation, ['type'] )

In [464]:
print(validation_cat.shape)
print(validation_scat.shape)
print(validation_t.shape)

(300, 19)
(300, 112)
(300, 2)


In [465]:
from sklearn.decomposition import PCA

In [466]:
# pca = PCA(n_components=10)
# validation_cat = pd.DataFrame(pca.fit_transform(validation_cat))
# train_cat = pd.DataFrame(pca.fit_transform(train_cat))
# pca = PCA(n_components=50)
# validation_scat = pd.DataFrame(pca.fit_transform(validation_scat))
# train_scat = pd.DataFrame(pca.fit_transform(train_scat))

In [467]:
print(validation_cat.shape)
print(validation_scat.shape)
print(validation_t.shape)

(300, 19)
(300, 112)
(300, 2)


In [468]:
train = pd.concat([fe_train, train_cat, train_scat, train_t, y_train], axis=1)
validation = pd.concat([fe_validation, validation_cat, validation_scat, validation_t, y_validation], axis=1) 

print(train.shape)

(6251, 138)


In [469]:
train.head(3)

,languages,num_langs,reference,G_mean_pr,category_architecture,category_biology,category_books,category_comics and anime,category_fashion,category_films,...,subcategory_traditional costume,subcategory_transport,subcategory_transport company,subcategory_tree,subcategory_visual arts,subcategory_writer,subcategory_writing style,type_concept,type_entity,label
0,2,6,5,1.000000,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
1,8,30,40,0.004405,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2
2,3,4,1,0.244156,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2


In [470]:
validation.head(3)

,languages,num_langs,reference,G_mean_pr,category_architecture,category_biology,category_books,category_comics and anime,category_fashion,category_films,...,subcategory_traditional costume,subcategory_transport,subcategory_transport company,subcategory_tree,subcategory_visual arts,subcategory_writer,subcategory_writing style,type_concept,type_entity,label
0,10,62,40,0.264605,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2
1,4,7,4,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
2,10,68,166,0.002405,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2


## Network

In [471]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

### Agnostic-Representative classifier

In [472]:
# prepare the dataset in order to take only two classes and eliminates the labels of the elements
d = train.query("label == 0 or label == 1")
y = d['label'].astype(int).to_numpy()
x = d.drop(['label'], axis=1).astype(float).to_numpy()

In [473]:
ar_tree = MLPClassifier().fit(x, y)

/home/andrea/miniconda3/envs/MNLP/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [474]:
d = validation.query("label == 0 or label == 1")
y = d['label'].astype(int).to_numpy()
x = d.drop(['label'], axis=1).astype(float).to_numpy()

In [475]:
y_pred = ar_tree.predict(x)
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.94      0.93       117
           1       0.90      0.87      0.89        76

    accuracy                           0.91       193
   macro avg       0.91      0.90      0.91       193
weighted avg       0.91      0.91      0.91       193



## Agnostic-Exclusive Classifier

In [476]:
# prepare the dataset in order to take only two classes and eliminates the labels of the elements
d = train.query("label == 0 or label == 2")
y = d['label'].astype(int).to_numpy()
x = d.drop(['label'], axis=1).astype(float).to_numpy()

In [477]:
ae_tree = MLPClassifier().fit(x, y)

/home/andrea/miniconda3/envs/MNLP/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [478]:
d = validation.query("label == 0 or label == 2")
y = d['label'].astype(int).to_numpy()
x = d.drop(['label'], axis=1).astype(float).to_numpy()

In [479]:
y_pred = ae_tree.predict(x)
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.93      0.82       117
           2       0.89      0.63      0.74       107

    accuracy                           0.79       224
   macro avg       0.81      0.78      0.78       224
weighted avg       0.81      0.79      0.78       224



## Representative-Exclusive

In [480]:
# prepare the dataset in order to take only two classes and eliminates the labels of the elements
d = train.query("label == 1 or label == 2")
y = d['label'].astype(int).to_numpy()
x = d.drop(['label'], axis=1).astype(float).to_numpy()

In [481]:
from sklearn.ensemble import RandomForestClassifier

In [482]:
re_tree = RandomForestClassifier().fit(x, y)

In [483]:
d = validation.query("label == 1 or label == 2")
y = d['label'].astype(int).to_numpy()
x = d.drop(['label'], axis=1).astype(float).to_numpy()

In [484]:
y_pred = re_tree.predict(x)
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           1       0.71      0.82      0.76        76
           2       0.85      0.77      0.81       107

    accuracy                           0.79       183
   macro avg       0.78      0.79      0.78       183
weighted avg       0.80      0.79      0.79       183



# Voting Schema


In [491]:
from scipy.stats import mode
from sklearn.base import ClassifierMixin, BaseEstimator
from sklearn.ensemble import IsolationForest

# Cultural-classification Network
class CABNet(BaseEstimator, ClassifierMixin):
    
    def __init__(self, ar_estimator, ae_estimator, re_estimator) -> None:
        self.ar = ar_estimator 
        self.ae = ae_estimator 
        self.re = re_estimator
        self.decisor = MLPClassifier()
        
    
    def __fit(self, e, X, y) -> BaseEstimator:
        return e.fit(X, y)


    def fit(self, X, y=None):
        d = X.query("label == 0 or label == 1 ")
        y = d['label'].astype(int).to_numpy()
        x = d.drop(['label'], axis=1).astype(float).to_numpy()
        self.ar = self.__fit(self.ar, x, y)
        d = X.query("label == 0 or label == 2")
        y = d['label'].astype(int).to_numpy()
        x = d.drop(['label'], axis=1).astype(float).to_numpy()
        self.ae = self.__fit(self.ae, x, y)
        d = X.query("label == 1 or label == 2")
        y = d['label'].astype(int).to_numpy()
        x = d.drop(['label'], axis=1).astype(float).to_numpy()
        self.re = self.__fit(self.re, x, y)
        return self
        
    def predict(self, X):
        v1 = self.ar.predict(X)
        v2 = self.ae.predict(X)
        v3 = self.re.predict(X)
     
  
        votes = np.vstack([v1, v2, v3])
        majority, _ = mode(votes, axis=0)
        return majority.ravel()

In [492]:
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, IsolationForest
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier

In [493]:
model =  CABNet(RandomForestClassifier(n_estimators=100), RandomForestClassifier(n_estimators=500), RandomForestClassifier(n_estimators=500))

In [494]:
model = model.fit(train)

In [495]:
d = validation
y = d['label'].astype(int).to_numpy()
x = d.drop(['label'], axis=1).astype(float).to_numpy()

In [496]:
y_pred = model.predict(x)
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       0.74      0.85      0.79       117
           1       0.69      0.76      0.72        76
           2       0.75      0.58      0.65       107

    accuracy                           0.73       300
   macro avg       0.73      0.73      0.72       300
weighted avg       0.73      0.73      0.73       300

